# Ejercicio: Regresión Lineal vs SVM con el dataset penguins

Vamos a comparar dos modelos de Machine Learning usando el dataset penguins de Seaborn.

## Contexto del estudio: campaña científica en la Antártida

Imagina que formas parte de un equipo de biólogos en la Estación Palmer (Antártida).
Durante varias temporadas, el equipo ha medido a pingüinos de tres especies —Adélie, Chinstrap y Gentoo— que viven en tres islas (Biscoe, Dream y Torgersen). El objetivo es entender cómo varían sus características corporales y si esas diferencias permiten identificar la especie automáticamente.

A partir de esas expediciones tenemos el dataset penguins, con medidas sencillas tomadas en el campo.

## Columnas del dataset

Cada fila representa un pingüino y contiene estas variables:

### species
    Especie del pingüino: Adelie, Chinstrap, Gentoo.

### island
    Isla donde se tomó la medición: Biscoe, Dream, Torgersen.

### bill_length_mm
    Longitud del pico en milímetros.

### bill_depth_mm
    Profundidad del pico (grosor) en milímetros.

### flipper_length_mm
    Longitud de la aleta en milímetros.

### body_mass_g
    Masa corporal en gramos.

### sex
    Sexo del individuo: male / female.



# Objetivos:
1. **EDA**
2. **Regresión lineal** para estudiar la relación entre dos variables numéricas según la **especie**.
   - Calculamos **correlaciones por especie** y ajustamos una regresión lineal simple.
   - Mostramos **matriz de confusión** (train/test) y **métricas** (accuracy, precision, recall, specificity, F1).
3. **SVM** para clasificar la **especie** usando 2 features numéricas (fácil de visualizar).
   - Permite **jugar con `kernel`, `C`, `gamma`, `degree`**.
   - Mostramos **matriz de confusión** y **classification_report**.
   - Dibujamos **regiones de decisión** en 2D (una figura por modelo).


In [10]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from matplotlib.colors import ListedColormap
import seaborn as sns

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

plt.rcParams['figure.figsize'] = (7,5)
plt.rcParams['axes.grid'] = True


## 1) Carga de datos y EDA rápido
⚠️ **Obligatorio:** usar `sns.pairplot` para visualizar relaciones entre varias variables a la vez. Así como shape, dtypes, describe, value_counts de la librería Pandas

In [ ]:

penguins = sns.load_dataset('penguins')
cols_keep = ['species','bill_length_mm','flipper_length_mm','bill_depth_mm']
df = penguins[cols_keep].dropna().reset_index(drop=True) # Justifica por qué es buena práctica eliminar los valores nulos NA, analiza el EDA entero para concluir esto

print("Shape:", )
print(df.head())

# Descripción numérica (.describe())

# Conteo por especie (.value_counts())
print("\nConteo por especie:")


# Dispersión bill_length vs flipper_length, por especie (m)

# isna()
print("\nValores nulos por columna:")

#sns.pairplot
sns.pairplot("", vars=[], hue='')
plt.show()


####### No tocar
species = df['species'].unique()
markers = ['o','s','^']
plt.figure()
for sp, mk in zip(species, markers):
    mask = df['species']==sp
    plt.scatter(df.loc[mask,'bill_length_mm'],
                df.loc[mask,'flipper_length_mm'],
                marker=mk, alpha=0.7, label=sp, edgecolor='k')
plt.xlabel('bill_length_mm'); plt.ylabel('flipper_length_mm')
plt.title('Dispersión por especie')
plt.legend()
plt.show()


## 2) Split 70/30 (Regresión)

In [ ]:
# Elige las variables de REGRESIÓN (cámbialas libremente):
X_VAR = 'bill_length_mm'       # predictor (eje X)
Y_VAR = 'flipper_length_mm'    # objetivo continuo (eje Y)

X_all = df[[X_VAR]].values
y_all = df[Y_VAR].values
g_all = df['species'].values

X_tr, X_te, y_tr, y_te, g_tr, g_te = train_test_split(
    X_all, y_all, g_all, test_size=0.30, random_state=42, stratify=g_all
)


## 3) Regresión lineal: relación **flipper_length_mm => bill_length_mm**
- Calculamos la **correlación (Pearson)** por especie para ver diferencias.
- Ajustamos una **regresión lineal simple** sobre *train*.


In [ ]:
# Pequeño helper SOLO para graficar una recta de regresión y el scatter | NO TOCAR
def plot_reg_line(ax, X, y, pipe, title):
    ax.scatter(X.ravel(), y, alpha=0.75, edgecolor='k', label='Datos')
    xs = np.linspace(X.min(), X.max(), 200).reshape(-1, 1)
    ys = pipe.predict(xs)
    ax.plot(xs, ys, lw=2, label='Recta')
    ax.set_xlabel(X_VAR); ax.set_ylabel(Y_VAR); ax.set_title(title)
    ax.legend()

# ========================================
# Variable de la especie a observar (cámbiala para observar otras especies)
sp = 'Adelie'
mask_tr = (g_tr == sp)
mask_te = (g_te == sp)

X_tr_sp, y_tr_sp = X_tr[mask_tr], y_tr[mask_tr]
X_te_sp, y_te_sp = X_te[mask_te], y_te[mask_te]

reg_adelie = Pipeline([('scaler', StandardScaler()), ('lr', LinearRegression())])
reg_adelie.fit(X_tr_sp, y_tr_sp)

yhat_tr = reg_adelie.predict(X_tr_sp)
yhat_te = reg_adelie.predict(X_te_sp)

print('=== Regresión lineal ===')
print(f'Tamaño train/test: {len(y_tr_sp)} / {len(y_te_sp)}')
print('MSE (train/test):', round(mean_squared_error(y_tr_sp, yhat_tr),3), '/',
      round(mean_squared_error(y_te_sp, yhat_te),3))
print('MAE (train/test):', round(mean_absolute_error(y_tr_sp, yhat_tr),3), '/',
      round(mean_absolute_error(y_te_sp, yhat_te),3))
print('R²  (train/test):', round(r2_score(y_tr_sp, yhat_tr),3), '/',
      round(r2_score(y_te_sp, yhat_te),3))

fig, axes = plt.subplots(1, 2, figsize=(10,4))
plot_reg_line(axes[0], X_tr_sp, y_tr_sp, reg_adelie, 'TRAIN')
plot_reg_line(axes[1], X_te_sp, y_te_sp, reg_adelie, 'TEST')
plt.tight_layout(); plt.show()




## 4) Split 70/30 (estratificado por especie)

In [14]:
# ===== 1) ELIGE LAS FEATURES PARA CLASIFICAR species (cámbialas libremente)
FEAT_X = 'bill_length_mm'
FEAT_Y = 'flipper_length_mm'

Xs = df[[FEAT_X, FEAT_Y]].values
ys = df['species'].values

Xs_tr, Xs_te, ys_tr, ys_te = train_test_split(
    Xs, ys, test_size=0.30, random_state=RANDOM_STATE, stratify=ys
)


## 4) SVM para clasificar la **especie** con 2 features
Usaremos **bill_length_mm** y **flipper_length_mm** como predictores para una clasificación *multiclase* (3 especies).  
Modifica fácilmente estos hiperparámetros y re‑ejecuta:


In [ ]:
# ===== 2) HELPERS (NO TOCAR): gráfica de CM y regiones de decisión
def plot_cm(cm, classes, title):
    fig, ax = plt.subplots()
    im = ax.imshow(cm, cmap='Blues')
    ax.set_xticks(np.arange(len(classes))); ax.set_yticks(np.arange(len(classes)))
    ax.set_xticklabels(classes); ax.set_yticklabels(classes)
    ax.set_xlabel('Predicción'); ax.set_ylabel('Real')
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, int(cm[i, j]), ha='center', va='center', color='black')
    plt.colorbar(im, ax=ax)
    plt.title(title); plt.tight_layout(); plt.show()

def plot_decision_regions_2d(model, X, y, title):
    # Malla 2D
    x_min, x_max = X[:,0].min()-1.0, X[:,0].max()+1.0
    y_min, y_max = X[:,1].min()-1.0, X[:,1].max()+1.0
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500),
                         np.linspace(y_min, y_max, 500))
    grid = np.c_[xx.ravel(), yy.ravel()]
    # Predicción en la malla (labels → enteros para colorear)
    classes = model.classes_
    mapping = {cls:i for i,cls in enumerate(classes)}
    Z_lab = model.predict(grid).reshape(xx.shape)
    Z = np.vectorize(mapping.get)(Z_lab)
    y_num = np.vectorize(mapping.get)(y)
    # Paleta clara para 3 clases (diferenciable en proyector)
    cmap = ListedColormap(['#1f77b4', '#d62728', '#2ca02c'])  # azul/rojo/verde
    plt.figure(figsize=(6.5,5))
    plt.contourf(xx, yy, Z, levels=np.arange(len(classes)+1)-0.5, cmap=cmap, alpha=0.25)
    plt.scatter(X[:,0], X[:,1], c=y_num, cmap=cmap, edgecolor='k', alpha=0.9)
    plt.xlabel(FEAT_X); plt.ylabel(FEAT_Y)
    plt.title(title + f"  (clases: {classes.tolist()})")
    plt.tight_layout(); plt.show()

# ============================================================
# TOMA 1 — SVM LINEAL (punto de partida recomendado)
# Objetivo: ver una frontera “plana” y entender el papel de C.
# Sugerencia: cambia C (0.1, 1, 10, 100) y prueba diferentes opciones
# ============================================================
svm_linear = Pipeline([
    ('scaler', StandardScaler()),
    ('svc',    SVC(kernel='linear', C=1.0, random_state=RANDOM_STATE))
])
svm_linear.fit(Xs_tr, ys_tr)

pred_tr = svm_linear.predict(Xs_tr)
pred_te = svm_linear.predict(Xs_te)

print("=== SVM lineal ===")
print("Parámetros: kernel='linear', C=1.0")
print("Accuracy (train/test):", round(accuracy_score(ys_tr, pred_tr),3), "/", round(accuracy_score(ys_te, pred_te),3))

cm_tr = confusion_matrix(ys_tr, pred_tr, labels=svm_linear.classes_)
cm_te = confusion_matrix(ys_te, pred_te, labels=svm_linear.classes_)
plot_cm(cm_tr, svm_linear.classes_, "Confusión — TRAIN (SVM lineal)")
plot_cm(cm_te, svm_linear.classes_, "Confusión — TEST  (SVM lineal)")

print("Classification report (TEST):\n", classification_report(ys_te, pred_te, digits=3))
plot_decision_regions_2d(svm_linear, np.r_[Xs_tr, Xs_te], np.r_[ys_tr, ys_te],
                         "SVM lineal — regiones de decisión")

# ============================================================
# Haz lo mismo para un SVM polinómico y RBF, crea la toma 2, 3 para ello. Consulta los recursos del campus si tienes dudas sobre ellos.
# ============================================================


## 5) Preguntas para entregar
1. ¿Qué **especie** muestra mayor **correlación** entre `bill_length_mm` y `flipper_length_mm`? ¿Existe otra relación entre variables y especie?
2. En el **SVM**, prueba al menos **tres combinaciones** de hiperparámetros:  
   - `kernel='linear'`  
   - `kernel='rbf'` con `C` alto y `gamma` pequeño  
   - `kernel='poly'` con `degree=3`  
   Explica **cómo cambia la matriz de confusión** y qué combinación recomendarías y por qué.
